## 1 Working with Extracellular Spike Data

The goal of this assignment is for you to learn how extracellular neural recordings are analyzed. You will be looking at recordings obtained from the songbird auditory cortex while recordings of bird songs were played to the animals.

![experiment diagram](images/experiment_diagram.png "Auditory Neurophysiology Experiment")

This web page is a Python notebook. It's a interactive document that lets you mix text and computer code, organized into `cells`. A cell can contain either text or code. This is a text cell. You can edit the contents of this cell by double-clicking, then save by typing `Ctrl-Enter`.

**Your Assignment**: Turn in a written response to each of the questions (e.g. **Q1**) below.

The cell below this is a code cell, as indicated by `In [ ]:` off to the left. When you type `Ctrl-Enter` in a code cell, the contents are passed to the Python interpreter. Each line is evaluated in turn. If the last line generates some output, this will be shown below the code cell. To run the notebook, you need to type `Ctrl-Enter` in each cell (including the one below):

In [ ]:
# These lines of code import some external libraries we'll use in this exercise. Hit Ctrl-Enter to execute.
%matplotlib notebook
from tools import signal, pprox, graphics
import numpy as np
import IPython
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.aspect'] = 'auto'
mpl.rcParams['image.cmap'] = 'jet'
mpl.rcParams['figure.figsize'] = (15.0, 4.0)

### Auditory responses

The basic structure of an auditory neurophysiology experiment is that we play a stimulus to the animal while recording from one or more neurons (or *units*) in the brain. The neurons are usually presented with a range of different stimuli to probe how selective they are or determine what parts of the stimulus excite them the best.

Let's look at the response of a neuron to a stimulus:

In [ ]:
%matplotlib notebook
stim_name = "A8"
stim, sampling_rate = signal.load_stimulus(stim_name)
spec, freq, bins = signal.specgram(stim, sampling_rate)
r, t = signal.load_raw_responses("st11_1_2", stim_name)
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 4), sharex=True)
axes[0].imshow(spec, extent=(bins[0], bins[-1], freq[0], freq[-1]))
axes[0].set_ylabel("Frequency (kHz)")
axes[0].set_title("Stimulus spectrogram")
axes[1].plot(t, r[0])
axes[1].set_xlabel("Time (s)")
axes[1].set_title("Response");

The plot generated above is *interactive*. Click the icon that looks like a square, and you can zoom in on different regions of the top or bottom graph. To restore the original view, click the icon that looks like a house.

The top plot is a **spectrogram** of the stimulus. Spectrograms show the power in the sound as a function of time and frequency. More intense colors indicate more power. You can read the plot like a musical score.

The bottom plot shows the neural recording. The recording begins before the stimulus starts and continues after it ends. These parts of the response are called the **background** or **spontaneous activity**. The part of the response that's aligned with the stimulus is called the **evoked response**.

**Q1**

1. Most of the recording is low-amplitude **noise**. Zoom in on a section of the response around 0 s. Describe what this looks like up close.
3. This neuron responded with brief bursts of **action potentials**, which caused high-amplitude **spikes** in the signal around 4 s. Zoom in on these and describe what the spikes look like.
4. Zoom back out a little and see if you can spot some patterns in the spectrogram that preceded the spiking. Do these data suggest that the neuron is responding to a particular kind of sound?

---
There are many sources of variability in the brain, so not every spike in a given trial is necessarily caused by the stimulus. Thus, each stimulus is usually presented 5-20 times to get an average that represents the part of the response that's driven by the stimulus. Let's look at a few responses to the same stimulus:

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(10, 4), sharex=True)
axes[0].imshow(spec, extent=(bins[0], bins[-1], freq[0], freq[-1]))
axes[0].set_ylabel("Frequency (kHz)")
axes[1].plot(t, r[0])
axes[2].plot(t, r[1])
axes[3].plot(t, r[2])
axes[3].set_xlabel("Time (s)")

**Q2:** Zoom in on the spike bursts and compare the responses across the trials. What are the differences? What stays the same?

### Spike sorting

A key first step in analyzing the results of an experiment is to *sort spikes*. This is a process that takes the raw neural recordings, which are densely sampled time series, and extracts the times when a spike ocurred. To be able to say that a set of spikes represents a single neuron, we need to make sure that the waveforms are distinct from the noise. There are many different methods of doing *spike sorting*, but the major steps usually consist of:

1. filter the recording to emphasize fast transients (spikes)
1. identify potential spikes and extract their waveforms
2. cluster similar waveforms together and exclude noise and artifacts

![spike sorting diagram](images/spike_sorting_diagram.png)

Let's start with spike detection. What we need to do is determine when the signal crosses above a threshold. Setting the threshold is usually done manually. The following code snippet will illustrate how this is done.

In [ ]:
from tools import graphics

fig = plt.figure(figsize=(10,4))
ax1 = plt.subplot2grid((1,4),(0,0),colspan=3)
ax1.set_title("Response")
ax2 = plt.subplot2grid((1,4),(0,3))
ax2.set_title("Spikes")
fig.subplots_adjust(wspace=0.3)

# change unit name and stimulus to try with other data
r, t = signal.load_raw_responses("st11_1_2", "A8")
graphics.spike_detector(ax1, ax2, r[0], t)

Move the `Threshold` slider to adjust the position of the black line. Any events that cross this line will be detected. A subset of the detected events will be plotted in the right panel as you adjust the threshold, and you can see the detected events marked with a red dot.

**Q3:** Try adjusting the threshold so that all of the detected events have similar waveforms. How many spikes were evoked by the stimulus? What happens if you set the threshold too low?

---
We're now going to cluster the spikes. The method we'll use is based on **principal components analysis**, which converts each waveform into a set of several numbers that describe its shape. In spike sorting, you're going to select a cluster of waveforms that correspond to the spikes. There are other waveforms that correspond to noise.

Switch to the `klusters` window. Your instructor has already opened up the data corresponding to the `st11_1_2` unit. Below is an annotated version of what your window should look like:

![spike sorting diagram](images/klusters_start.png)

There are several different panels:

- The spike features panel shows a scatter plot of the events in two dimensions. Each event is shown as a dot with the position determined by several different measurements (features). You can cycle through the different features by changing the values in the `Features (x,y)` control.
- The spike waveforms panel shows a selection of waveforms in each of the non-artifact clusters, similar to the right panel in the figure above.
- The autocorrelogram panel shows the distribution of intervals between events. This plot will be used later to validate your clustering.

Select the `draw new cluster` tool (looks like a green sphere with a `+` next to it). Draw a polygon around one of the clusters by clicking with the left mouse button. When you're ready to close the polygon, click the center mouse button. Once you close the polygon a new cluster will form. Each cluster receives a different color. Look in the waveform panel to see what the spikes in that cluster look like, and compare them to the events that are left over (the `1` cluster). Which cluster corresponds to the spikes?

**Q4:** Each waveform has 7 different features associated with it. One of these features corresponds to time. Cycle through the variables by clicking the up and down arrows next to `Features` in the tool bar. Which dimension looks like it relates to time? Which dimension do you think is associated with the amplitude of the spike?

---
Now we're going to make sure this group of spikes corresponds to a **single unit**. To make sure that there are only a few (if any) **refractory period violations**, we're going to look at the **autocorrelation plot** (below). Note that your screen may look different if you clustered out the smaller events.

![spike sorting diagram](images/klusters_split_acor.png)

The autocorrelogram is a histogram of all the intervals between spike times. The center corresponds to zero and each bin is 1 ms. If the unit is well isolated, you should see a deep trough in the center, ideally with all the bins <= 1 ms having zero spikes. Compare this to the autocorrelogram for the noise cluster.

You can merge clusters and split them to see how your selections influence the autocorrelogram and waveforms. Hover your mouse over the tools to see what each one does.

**Q5:** Try creating a cluster that includes dots from both groups in the features panel. What does the autocorrelogram look like for this cluster? How does the autocorrelogram tell you this is not a good cluster?

---
Save your work, then click `Open` from the `File` menu and select another unit from the list below. Sort the spikes. Show your work to an instructor, then proceed to sort the rest of the list.

```
st348_2_7.fet.1
st348_4_1.fet.3
st11_1_2.fet.1
st268_2_1.fet.3
st268_5_2.fet.1
st49_4_2.fet.3
st418_4_2.fet.1
```

**Q6:** Two of the recordings captured two or more units. Which ones? How are the two units different from each other? How can you tell that they are two different single units rather than just a weirdly shaped cluster?

---
### Visualizing spike time data

We're going to skip one step in the process, which is to take the sorted spikes and write the spike times to a file. This has already been done, so you can now open up the resulting file and plot the spike time data as a **raster plot**. Run the code snippet below:

In [ ]:
from tools import pprox
unit = "st11_1_2_1"
stim_name = "A8"

stim, sampling_rate = signal.load_stimulus(stim_name)
spec, freq, bins = signal.specgram(stim, sampling_rate)

resp = pprox.load(unit)
resp_A8 = pprox.select_stimulus(resp, stim_name)
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 6), sharex=True)
axes[0].imshow(spec, extent=(bins[0], bins[-1], freq[0], freq[-1]))
axes[0].set_ylabel("Frequency (kHz)")
axes[0].set_title("Stimulus spectrogram")
axes[1].plot(t, r[0])
axes[1].set_title("Raw response - Trial 1")
graphics.plot_raster(axes[2], resp_A8)
axes[2].set_xlabel("Time (s)")
axes[2].set_title("Raster");

**Q7:** How does this plot compare to what you saw in the raw recordings? How are the spikes represented in the raster plot? What's the advantage of visualizing the response this way?

---
Now let's look at the response to several different stimuli.

In [ ]:
unit = "st11_1_2_1"
resp = pprox.load(unit)
stims = ["A8", "B8", "C8"]
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10, 4), sharex='col')
for i, stim in enumerate(stims):
    osc, sampling_rate = signal.load_stimulus(stim)
    spec, freq, bins = signal.specgram(osc, sampling_rate)
    axes[0, i].imshow(spec, extent=(bins[0], bins[-1], freq[0], freq[-1]))
    axes[0, i].set_title(stim)
    graphics.plot_raster(axes[1, i], pprox.select_stimulus(resp, stim))
    axes[1, i].set_xlabel("Time (s)")


**Q8:** Zoom in to the parts of the spectrogram that evoked strong responses. Do you see any patterns?

---
Pick one of the units from this list. Paste it into the code snippet **above**, between the quotes on the line that starts with `unit =`. Pick another unit to try and do the same.

```
st348_2_7_2
st348_4_1_3
st268_2_1_4
st268_5_2_1
st49_4_2_3
st418_4_2_1
st376_1_1_6
```

**Q9:** How do the responses of these units and `st11_1_2_1` compare to each other? Discuss this in terms of spontaneous and evoked activity, and how selective or nonselective the unit appears to be.